# Smets & Wouters (2007) - Replicacion de Figuras 1 y 2

Este notebook replica las Figuras 1 y 2 del paper "Shocks and Frictions in US Business Cycles: A Bayesian DSGE Approach" (AER, 2007).

**Figura 1**: Forecast Error Variance Decomposition (FEVD)
- Descomposicion de varianza del error de pronostico para GDP growth, Inflation y Interest rate
- Muestra la contribucion de cada shock estructural a diferentes horizontes

**Figura 2**: Impulse Response Functions (IRFs) a shocks de demanda
- Respuestas de Output y Hours worked a shocks de demanda
- Shocks: Risk premium (eb), Government spending (eg), Investment-specific (eqs)

**Requisitos previos:**
- GNU Octave instalado y en PATH
- Dynare instalado
- Archivo `usmodel_mode.mat` con posterior mode

## 1. Setup y Configuracion

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
from pathlib import Path

# Add parent directory to path
sys.path.append(str(Path.cwd().parent.parent))

from direct_replication import DynareInterface

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("Imports completados")

In [ ]:
# Configurar rutas - MODIFICAR SEGUN TU INSTALACION

# Configurar ruta de Octave (necesario en Windows)
os.environ['OCTAVE_EXECUTABLE'] = r'C:\Program Files\GNU Octave\Octave-10.3.0\mingw64\bin\octave-cli.exe'

DYNARE_PATH = r'C:\dynare\6.5\matlab'  # Ruta a carpeta matlab de Dynare
REPO_PATH = Path.cwd().parent.parent / 'repo'  # Carpeta con datos (.xls)
MODEL_PATH = Path.cwd().parent / 'model'  # Carpeta con .mod files

print(f"Octave executable: {os.environ['OCTAVE_EXECUTABLE']}")
print(f"Dynare path: {DYNARE_PATH}")
print(f"Repo path: {REPO_PATH}")
print(f"Model path: {MODEL_PATH}")
print(f"Repo exists: {REPO_PATH.exists()}")
print(f"Model exists: {MODEL_PATH.exists()}")

### Test de Conexion Octave + Dynare

In [ ]:
from oct2py import Oct2Py

# Test basico
try:
    oc = Oct2Py()
    result = oc.eval('2 + 2', nout=1)
    print(f"Octave conectado: 2 + 2 = {result}")
    
    # Test Dynare
    oc.addpath(DYNARE_PATH)
    dynare_exists = oc.eval('exist("dynare")', nout=1)
    if dynare_exists == 2:
        print("Dynare encontrado")
    else:
        print("Dynare NO encontrado - verificar DYNARE_PATH")
    
    oc.exit()
    print("\nTodo listo!")
    
except Exception as e:
    print(f"Error: {e}")
    print("Ver setup_instructions.md para solucion de problemas")

## 2. Datos de Referencia del Paper

### Figura 1: Forecast Error Variance Decomposition (FEVD)
Muestra como los 7 shocks estructurales contribuyen a la varianza del error de pronostico en diferentes horizontes para:
- GDP Growth (dy)
- Inflation (pinfobs)
- Interest Rate (robs)

### Figura 2: IRFs a Shocks de Demanda
Muestra las respuestas de Output (y) y Hours (lab) a los 3 shocks de "demanda":
- Risk premium shock (eb)
- Government spending shock (eg)
- Investment-specific shock (eqs)

In [ ]:
# Definiciones para las figuras

# Labels para los 7 shocks estructurales
SHOCK_LABELS = {
    'ea': 'Productivity',
    'eb': 'Risk premium',
    'eg': 'Exog. spending',
    'eqs': 'Investment',
    'em': 'Monetary',
    'epinf': 'Price markup',
    'ew': 'Wage markup'
}

SHOCK_ORDER = ['ea', 'eb', 'eg', 'eqs', 'em', 'epinf', 'ew']

# Figura 1: Variables para FEVD
FIGURE1_VARIABLES = ['dy', 'pinfobs', 'robs']
FIGURE1_LABELS = {
    'dy': 'GDP Growth',
    'pinfobs': 'Inflation',
    'robs': 'Federal Funds Rate'
}

# Figura 2: Variables y shocks de demanda
DEMAND_SHOCKS = ['eb', 'eg', 'eqs']
FIGURE2_VARIABLES = ['y', 'lab']
FIGURE2_LABELS = {
    'y': 'Output',
    'lab': 'Hours'
}

print("Configuracion de figuras definida")
print(f"\nFigura 1 - FEVD para: {FIGURE1_VARIABLES}")
print(f"Figura 2 - IRFs de {FIGURE2_VARIABLES} a shocks: {DEMAND_SHOCKS}")

## 3. Ejecutar Dynare con Modelo Modificado

Usamos `usmodel_figures.mod` que tiene el comando `stoch_simul` modificado para generar:
- Conditional Variance Decomposition (FEVD) para horizontes 1-40
- IRFs para variables adicionales (y, lab)

In [ ]:
# Inicializar interfaz Dynare
di = DynareInterface(DYNARE_PATH, str(MODEL_PATH))

print("Interfaz Dynare inicializada")

In [ ]:
# Ejecutar modelo modificado para figuras
# Este modelo incluye conditional_variance_decomposition y variables adicionales

print("Ejecutando usmodel_figures.mod...")
print("(Esto puede tardar varios minutos)\n")

di.run_model('usmodel_figures.mod')

print("\nDynare completado")

## 4. Extraer FEVD para Figura 1

La Conditional Variance Decomposition muestra como cada shock contribuye a la varianza del error de pronostico en diferentes horizontes.

In [ ]:
def extract_fevd(di, max_horizon=40):
    """
    Extrae la Conditional Variance Decomposition de Dynare.
    
    Returns:
        DataFrame con columnas: horizon, variable, shock, variance_share
    """
    # Verificar que existe
    has_fevd = di.oc.eval('isfield(oo_, "conditional_variance_decomposition")', nout=1)
    if not has_fevd:
        raise RuntimeError("FEVD no encontrada. Verificar stoch_simul con conditional_variance_decomposition.")
    
    # Obtener dimensiones del array
    fevd_size = di.oc.eval('size(oo_.conditional_variance_decomposition)', nout=1)
    n_horizons = int(fevd_size[0])
    n_vars = int(fevd_size[1])
    n_shocks = int(fevd_size[2])
    
    print(f"FEVD dimensions: {n_horizons} horizons x {n_vars} variables x {n_shocks} shocks")
    
    # Obtener nombres de variables endogenas
    n_endo = int(di.oc.eval('M_.endo_nbr', nout=1))
    var_names = []
    for i in range(n_endo):
        name = di.oc.eval(f'deblank(M_.endo_names{{{i+1}}})', nout=1)
        var_names.append(str(name).strip())
    
    # Obtener nombres de shocks
    n_exo = int(di.oc.eval('M_.exo_nbr', nout=1))
    shock_names = []
    for i in range(n_exo):
        name = di.oc.eval(f'deblank(M_.exo_names{{{i+1}}})', nout=1)
        shock_names.append(str(name).strip())
    
    print(f"Variables: {var_names[:10]}...")
    print(f"Shocks: {shock_names}")
    
    # Extraer array FEVD
    fevd_array = di.oc.eval('oo_.conditional_variance_decomposition', nout=1)
    
    # Convertir a DataFrame
    data = []
    for h in range(min(n_horizons, max_horizon)):
        for v in range(n_vars):
            for s in range(n_shocks):
                # FEVD esta en porcentaje (0-100)
                value = fevd_array[h, v, s]
                data.append({
                    'horizon': h + 1,
                    'variable': var_names[v],
                    'shock': shock_names[s],
                    'variance_share': float(value)
                })
    
    return pd.DataFrame(data)

# Extraer FEVD
print("Extrayendo FEVD...\n")
fevd_df = extract_fevd(di, max_horizon=40)

print(f"\nFEVD extraida: {len(fevd_df)} filas")
fevd_df.head(10)

In [ ]:
# Verificar que FEVD suma 100% para cada (variable, horizonte)
print("Verificando que FEVD suma 100% para cada variable y horizonte...\n")

for var in FIGURE1_VARIABLES:
    var_data = fevd_df[fevd_df['variable'] == var]
    for h in [1, 4, 8, 20, 40]:
        h_data = var_data[var_data['horizon'] == h]
        total = h_data['variance_share'].sum()
        status = "OK" if abs(total - 100) < 0.5 else "WARNING"
        print(f"  {var} h={h}: {total:.2f}% [{status}]")

print("\nVerificacion completada")

## 5. Extraer IRFs para Figura 2

Extraemos las IRFs de Output (y) y Hours (lab) a los shocks de demanda.

In [ ]:
# Extraer IRFs
print("Extrayendo IRFs...\n")

irfs_df = di.get_irfs(periods=20)

print(f"IRFs extraidas: {len(irfs_df)} observaciones")
print(f"\nVariables disponibles: {irfs_df['variable'].unique()}")
print(f"Shocks disponibles: {irfs_df['shock'].unique()}")

irfs_df.head(10)

In [ ]:
# Filtrar IRFs para Figura 2
figure2_irfs = irfs_df[
    (irfs_df['variable'].isin(FIGURE2_VARIABLES)) & 
    (irfs_df['shock'].isin(DEMAND_SHOCKS))
].copy()

print(f"IRFs para Figura 2: {len(figure2_irfs)} observaciones")
print(f"\nVariables: {figure2_irfs['variable'].unique()}")
print(f"Shocks: {figure2_irfs['shock'].unique()}")

figure2_irfs.head()

## 6. Graficar Figura 1: FEVD

Creamos un grafico de areas apiladas (stacked area chart) para cada variable, mostrando la contribucion de cada shock a la varianza del error de pronostico.

In [ ]:
def plot_figure1_fevd(fevd_df, variables, shock_order, shock_labels, var_labels, max_horizon=40):
    """
    Grafica Figura 1: FEVD como stacked area charts.
    
    Estilo del paper:
    - 3 paneles (GDP growth, Inflation, Interest rate)
    - Eje X: Horizonte (quarters)
    - Eje Y: Porcentaje de varianza (0-100%)
    - Areas apiladas para cada shock
    """
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Colores para los shocks
    colors = plt.cm.Set2(np.linspace(0, 1, len(shock_order)))
    
    for idx, var in enumerate(variables):
        ax = axes[idx]
        
        # Filtrar para esta variable
        var_data = fevd_df[(fevd_df['variable'] == var) & (fevd_df['horizon'] <= max_horizon)].copy()
        
        # Pivot a formato wide
        pivot_data = var_data.pivot(
            index='horizon', 
            columns='shock', 
            values='variance_share'
        )
        
        # Reordenar columnas segun shock_order
        available_shocks = [s for s in shock_order if s in pivot_data.columns]
        pivot_data = pivot_data[available_shocks]
        
        # Grafico de area apilada
        pivot_data.plot.area(
            ax=ax,
            stacked=True,
            alpha=0.85,
            color=colors[:len(available_shocks)],
            linewidth=0.5
        )
        
        ax.set_title(var_labels.get(var, var), fontsize=12, fontweight='bold')
        ax.set_xlabel('Horizon (quarters)', fontsize=10)
        ax.set_ylabel('Percent', fontsize=10)
        ax.set_xlim(1, max_horizon)
        ax.set_ylim(0, 100)
        
        # Leyenda solo en el ultimo panel
        if idx == len(variables) - 1:
            ax.legend(
                [shock_labels.get(s, s) for s in available_shocks],
                loc='center left',
                bbox_to_anchor=(1.02, 0.5),
                fontsize=9
            )
        else:
            ax.legend().remove()
        
        ax.grid(True, alpha=0.3, linestyle='--')
    
    plt.suptitle('Figure 1: Forecast Error Variance Decomposition', fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    
    return fig

# Graficar Figura 1
print("Generando Figura 1: FEVD...\n")

fig1 = plot_figure1_fevd(
    fevd_df, 
    FIGURE1_VARIABLES, 
    SHOCK_ORDER, 
    SHOCK_LABELS, 
    FIGURE1_LABELS,
    max_horizon=40
)

plt.show()
print("\nFigura 1 generada")

## 7. Graficar Figura 2: IRFs a Shocks de Demanda

Creamos un grid 2x3 mostrando las respuestas de Output y Hours a los 3 shocks de demanda.

In [ ]:
def plot_figure2_irfs(irfs_df, variables, shocks, shock_labels, var_labels):
    """
    Grafica Figura 2: IRFs a shocks de demanda.
    
    Estilo del paper:
    - 2 filas (Output, Hours)
    - 3 columnas (Risk premium, Government, Investment)
    - Line plots con referencia en y=0
    """
    fig, axes = plt.subplots(2, 3, figsize=(12, 8))
    
    for i, var in enumerate(variables):
        for j, shock in enumerate(shocks):
            ax = axes[i, j]
            
            # Filtrar datos
            irf_data = irfs_df[
                (irfs_df['variable'] == var) & 
                (irfs_df['shock'] == shock)
            ].sort_values('period')
            
            if len(irf_data) > 0:
                # Grafico de linea
                ax.plot(irf_data['period'], irf_data['value'], 'b-', linewidth=2)
                ax.fill_between(irf_data['period'], 0, irf_data['value'], alpha=0.2)
            
            # Linea de referencia en y=0
            ax.axhline(0, color='k', linestyle='--', linewidth=0.8)
            
            # Titulos
            if i == 0:
                ax.set_title(shock_labels.get(shock, shock), fontsize=11, fontweight='bold')
            if j == 0:
                ax.set_ylabel(var_labels.get(var, var), fontsize=11, fontweight='bold')
            if i == 1:
                ax.set_xlabel('Quarters', fontsize=10)
            
            ax.grid(True, alpha=0.3, linestyle='--')
            ax.set_xlim(0, 20)
    
    plt.suptitle('Figure 2: Estimated Mean Impulse Responses to Demand Shocks', 
                 fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    
    return fig

# Graficar Figura 2
print("Generando Figura 2: IRFs a shocks de demanda...\n")

fig2 = plot_figure2_irfs(
    figure2_irfs,
    FIGURE2_VARIABLES,
    DEMAND_SHOCKS,
    SHOCK_LABELS,
    FIGURE2_LABELS
)

plt.show()
print("\nFigura 2 generada")

## 8. Exportar Resultados

In [ ]:
# Guardar figuras
output_dir = Path.cwd()

fig1.savefig(output_dir / 'figure1_fevd.png', dpi=300, bbox_inches='tight')
print(f"Figura 1 guardada en: {output_dir / 'figure1_fevd.png'}")

fig2.savefig(output_dir / 'figure2_irfs.png', dpi=300, bbox_inches='tight')
print(f"Figura 2 guardada en: {output_dir / 'figure2_irfs.png'}")

# Guardar datos en CSV
fevd_figure1 = fevd_df[fevd_df['variable'].isin(FIGURE1_VARIABLES)]
fevd_figure1.to_csv(output_dir / 'figure1_fevd_data.csv', index=False)
print(f"\nDatos FEVD guardados en: {output_dir / 'figure1_fevd_data.csv'}")

figure2_irfs.to_csv(output_dir / 'figure2_irfs_data.csv', index=False)
print(f"Datos IRFs guardados en: {output_dir / 'figure2_irfs_data.csv'}")

## 9. Resumen de Resultados

In [ ]:
# Resumen FEVD en horizontes clave
print("="*70)
print("RESUMEN: FEVD en horizontes seleccionados")
print("="*70)

for var in FIGURE1_VARIABLES:
    print(f"\n{FIGURE1_LABELS[var]}:")
    print("-" * 50)
    
    var_data = fevd_df[fevd_df['variable'] == var]
    
    for h in [1, 4, 10, 40]:
        h_data = var_data[var_data['horizon'] == h]
        print(f"  Horizon {h:2d}:  ", end="")
        for shock in SHOCK_ORDER:
            share = h_data[h_data['shock'] == shock]['variance_share'].values
            if len(share) > 0:
                print(f"{shock}={share[0]:5.1f}%  ", end="")
        print()

In [ ]:
# Resumen IRFs - valores en impacto y pico
print("\n" + "="*70)
print("RESUMEN: IRFs a shocks de demanda")
print("="*70)

for var in FIGURE2_VARIABLES:
    print(f"\n{FIGURE2_LABELS[var]}:")
    print("-" * 50)
    
    for shock in DEMAND_SHOCKS:
        irf_data = figure2_irfs[
            (figure2_irfs['variable'] == var) & 
            (figure2_irfs['shock'] == shock)
        ].sort_values('period')
        
        if len(irf_data) > 0:
            impact = irf_data[irf_data['period'] == 0]['value'].values[0]
            peak_idx = irf_data['value'].abs().idxmax()
            peak_period = irf_data.loc[peak_idx, 'period']
            peak_value = irf_data.loc[peak_idx, 'value']
            
            print(f"  {SHOCK_LABELS[shock]:15s}: Impact={impact:+.4f}, Peak={peak_value:+.4f} (t={int(peak_period)})")

In [ ]:
print("\n" + "="*70)
print("REPLICACION COMPLETADA")
print("="*70)
print("\nFiguras generadas:")
print("  - figure1_fevd.png: Forecast Error Variance Decomposition")
print("  - figure2_irfs.png: IRFs a shocks de demanda")
print("\nDatos exportados:")
print("  - figure1_fevd_data.csv")
print("  - figure2_irfs_data.csv")

## 10. Cleanup

In [ ]:
# Cerrar sesion Octave
di.close()
print("Sesion Octave cerrada")